# Get addres from location coordinates

## Import dataframe
This dataframe was created usin [get_gas_stations.py](/get_gas_stations.py) or [gasolinas_aio.ipynb](/gasolinas_aio.ipynb).

In [36]:
import pandas as pd

path_file = "export_files\gas_prices_20220415_160024.csv"
df = pd.read_csv(path_file)
df.set_index("place_id", inplace=True)
df

,name,cree_id,latitude,longitude,regular,premium,diesel
place_id,,,,,,,
2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,32.47641,-116.92140,22.49,24.99,21.99
2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,20.30370,-99.74484,21.25,23.19,23.09
2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,31.71947,-106.45140,17.28,19.20,NaN
2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,20.71413,-103.30420,21.79,24.39,23.54
2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,26.03787,-98.29977,17.79,19.99,NaN
...,...,...,...,...,...,...,...
27758,SUMINISTRO DE COMBUSTIBLES EL SEIS S. DE R.L. ...,PL/23961/EXP/ES/2022,20.88375,-102.70250,NaN,NaN,NaN
27759,GASOLINERA SACSALUM SA DE CV,PL/23960/EXP/ES/2022,16.32370,-91.97680,NaN,NaN,NaN
27764,"AEROPUERTO INTERNACIONAL FELIPE ANGELES, SOCIE...",PL/23974/EXP/ES/2022,19.72579,-99.03263,NaN,NaN,NaN


## Using geopy library
The code extracts multiple features from location coordinates using geopy library. The loop takes 1.5 h to get all address.

In [31]:
from geopy.geocoders import Nominatim
from time import time
import numpy as np

geolocator = Nominatim(user_agent="geoapiExercides")

[counter, wrongs] = [0,0]
indexs = df.dropna(subset=["latitude", "longitude"]).index
nplaces = len(indexs)

start_time = time()
estimate_time = 10

locations = []
for idx in indexs[4760:]:
    
    # Get locations
    location = geolocator.reverse(str(df.loc[idx,"latitude"])+","+str(df.loc[idx,"longitude"]))
    
    # If location has address, append to list
    if location:
        location = location.raw["address"]
        location["place_id"] = idx
        locations.append(location)
    else:
        wrongs += 1
    
    counter += 1

    # Calculate estimate remaining time
    if counter % 10 == 0:
        estimate_time = np.round(
            (time()-start_time)/(3600*counter)*(nplaces-counter), 4)
        
    # Print status process
    print('\rProcessed places {}/{}. Remaining time: {:5.4f} h'.format(counter,nplaces, estimate_time), end='')
    
print("Process finish with {} errors.".format(wrongs))

Processed places 12848/12851. Remaining time: 0.0010 hProcess finish with 3 errors.


## Creating pandas dataframe

In [32]:
address_df = pd.DataFrame(locations)
address_df.set_index("place_id", inplace=True)
address_df

,amenity,road,hamlet,city,county,state,ISO3166-2-lvl4,postcode,country,country_code,...,municipality,state_district,man_made,leisure,farm,railway,place,aeroway,craft,locality
place_id,,,,,,,,,,,,,,,,,,,,,
2039,Pemex,Boulevard de los Insurgentes,El Porvenir,Tijuana,Municipio de Tijuana,Baja California,MX-BCN,22223,México,mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2040,NaN,MEX 45,NaN,NaN,Nopala de Villagrán,Hidalgo,MX-HID,NaN,México,mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2041,Pemex E02526,Avenida Licenciado Adolfo López Mateos,NaN,Ciudad Juárez,Municipio de Juárez,Chihuahua,MX-CHH,94158,México,mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2042,Pemex,Anillo Periférico Norte Manuel Gómez Morín,NaN,Guadalajara,NaN,Jalisco,MX-JAL,44390,México,mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2043,NaN,NaN,NaN,Reynosa,Municipio de Reynosa,Tamaulipas,MX-TAM,88500,México,mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27758,NaN,Carretera Tepatitlán-Las Adjuntas,NaN,NaN,San Ignacio Cerro Gordo,Jalisco,MX-JAL,47601,México,mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27759,NaN,Chiapas 218,NaN,NaN,Las Margaritas,Chiapas,MX-CHP,30187,México,mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27764,NaN,Acceso a Aeropuerto Int. Felipe Angeles,NaN,NaN,Nextlalpan,Estado de México,MX-MEX,55789,México,mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Check NaN values for dataframe

In [33]:
address_df.isna().sum()

amenity               7335
road                  1210
hamlet               11797
city                  6824
county                1255
state                    8
ISO3166-2-lvl4           8
postcode              2224
country                  0
country_code             0
neighbourhood        10672
village              11036
residential          12193
town                  9306
building             12767
shop                 12264
house_number         12452
quarter              11877
suburb               12329
tourism              12812
industrial           12769
isolated_dwelling    12752
historic             12831
highway              12820
commercial           12824
office               12832
retail               12837
city_district        12667
municipality         12707
state_district       12794
man_made             12839
leisure              12821
farm                 12828
railway              12847
place                12846
aeroway              12847
craft                12847
l

## Drop unusefully columns
For project purposes we drop columns with a lot of NaN values.

In [55]:
address_df = address_df.iloc[:,0:8]
address_df

,amenity,road,hamlet,city,county,state,ISO3166-2-lvl4,postcode
place_id,,,,,,,,
2039,Pemex,Boulevard de los Insurgentes,El Porvenir,Tijuana,Municipio de Tijuana,Baja California,MX-BCN,22223
2040,NaN,MEX 45,NaN,NaN,Nopala de Villagrán,Hidalgo,MX-HID,NaN
2041,Pemex E02526,Avenida Licenciado Adolfo López Mateos,NaN,Ciudad Juárez,Municipio de Juárez,Chihuahua,MX-CHH,94158
2042,Pemex,Anillo Periférico Norte Manuel Gómez Morín,NaN,Guadalajara,NaN,Jalisco,MX-JAL,44390
2043,NaN,NaN,NaN,Reynosa,Municipio de Reynosa,Tamaulipas,MX-TAM,88500
...,...,...,...,...,...,...,...,...
27758,NaN,Carretera Tepatitlán-Las Adjuntas,NaN,NaN,San Ignacio Cerro Gordo,Jalisco,MX-JAL,47601
27759,NaN,Chiapas 218,NaN,NaN,Las Margaritas,Chiapas,MX-CHP,30187
27764,NaN,Acceso a Aeropuerto Int. Felipe Angeles,NaN,NaN,Nextlalpan,Estado de México,MX-MEX,55789


## Export DataFrame
The filename will have same name sintax as places file.

In [56]:
#Creating name of file based in gas_prices file
date_ex = path_file[-19:]

address_df.to_csv("export_files\gas_address_{}".format(date_ex))